 # 1. Постановка задачи

## 1.1 Описание проекта: промышленность

Чтобы оптимизировать производственные расходы, металлургический комбинат «Стальная птица» решил уменьшить потребление электроэнергии на этапе обработки стали. Для этого комбинату нужно контролировать температуру сплава. Ваша задача — построить модель, которая будет её предсказывать. 
Заказчик хочет использовать разработанную модель для имитации технологического процесса

## 1.2 Описание технологического процесса

Сталь обрабатывают в металлическом ковше вместимостью около 100 тонн. Чтобы ковш выдерживал высокие температуры, изнутри его облицовывают огнеупорным кирпичом. Расплавленную сталь заливают в ковш и подогревают до нужной температуры графитовыми электродами. Они установлены в крышке ковша. 
Из сплава выводится сера (этот процесс — десульфурация), добавлением примесей корректируется химический состав и отбираются пробы. Сталь легируют — изменяют её состав — подавая куски сплава из бункера для сыпучих материалов или проволоку через специальный трайб-аппарат (от англ. tribe — «масса»).
Перед тем как первый раз ввести легирующие добавки, измеряют температуру стали и производят её химический анализ. Потом температуру на несколько минут повышают, добавляют легирующие материалы и продувают сплав инертным газом. Затем его перемешивают и снова проводят измерения. Такой цикл повторяется до достижения целевого химического состава и оптимальной температуры плавки.
Тогда расплавленная сталь отправляется на доводку металла или поступает в машину непрерывной разливки. Оттуда готовый продукт выходит в виде заготовок-слябов (от англ. slab — «плита»).

## 1.3 Описание даннных

Данные состоят из нескольких файлов, полученных из разных источников:  
- data_arc_new.csv — данные об электродах;  
- data_bulk_new.csv — данные о подаче сыпучих материалов (объём);  
- data_bulk_time_new.csv — данные о подаче сыпучих материалов (время);  
- data_gas_new.csv — данные о продувке сплава газом;  
- data_temp_new.csv — результаты измерения температуры;  
- data_wire_new.csv — данные о проволочных материалах (объём);  
- data_wire_time_new.csv — данные о проволочных материалах (время)

Файл data_arc_new.csv

- key — номер партии;
- Начало нагрева дугой — время начала нагрева;
- Конец нагрева дугой — время окончания нагрева;
- Активная мощность — значение активной мощности;
- Реактивная мощность — значение реактивной мощности.

Файл data_bulk_new.cs:

- key — номер партии;  
- Bulk 1 … Bulk 15 — объём подаваемого материала.

Файл data_bulk_time_new.csv:
- key — номер партии;
- Bulk 1 … Bulk 15 — время подачи материала.

Файл data_gas_new.csv:
- key — номер партии;
- Газ 1 — объём подаваемого газа.

Файл data_temp_new.csv:

- key — номер партии;
- Время замера — время замера;
- Температура — значение температуры.

Файл data_wire_new.csv:

- key — номер партии;
- Wire 1 … Wire 15 — объём подаваемых проволочных материалов

Файл data_wire_time_new.csv

- key — номер партии;
- Wire 1 … Wire 15 — время подачи проволочных материалов.

Во всех файлах столбец key содержит номер партии. В файлах может быть несколько строк с одинаковым значением key: они соответствуют разным итерациям обработки.

## 1.4 Постановка задчи

Построить модель, которая предскажет температуру в системе "печь-ковш"(кислородного конвертера) при выплавке стали кислородно-конвертерным способом в конвертерном цеху металургического комбината

# 2. Импорт библиотек и датасета

## 2.1. Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.core.display import display, HTML;

/var/folders/5m/tbwsb3vs6734vbbw653my9v00000gp/T/ipykernel_14090/2959857078.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML;


In [2]:
display(HTML("<style>.container { width:90% !important; }</style>")) # Изменяем ширину тетради

## 2.2. Импорт датасета

In [3]:
def import_data(pth1:str, pth2:str) -> pd.DataFrame:
    try:
        df = pd.read_csv(pth1)
    except OSError as e:
         df = pd.read_csv(pth2)
    return df

In [4]:
pth1 = ['https://code.s3.yandex.net/datasets/data_arc_new.csv',
        'https://code.s3.yandex.net/datasets/data_bulk_new.csv',
        'https://code.s3.yandex.net/datasets/data_bulk_time_new.csv',
        'https://code.s3.yandex.net/datasets/data_gas_new.csv',
        'https://code.s3.yandex.net/datasets/data_temp_new.csv',
        'https://code.s3.yandex.net/datasets/data_wire_time_new.csv'
]

In [5]:
pth2 = ['/Documents/GitHub/final_project/data_arc_new.csv', 
        '/Documents/GitHub/final_project/data_bulk_new.csv',
        '/Documents/GitHub/final_project/data_bulk_time_new.csv',
        '/Documents/GitHub/final_project/data_gas_new.csv',
        '/Documents/GitHub/final_project/data_temp_new.csv',
        '/Documents/GitHub/final_project/data_wire_new.csv',
        '/Documents/GitHub/final_project/data_wire_time_new.csv'
]

In [6]:
arc_new = import_data(pth1[0], pth2[0]) #данные об электродах;
bulk_new = import_data(pth1[1], pth2[1]) #данные о подаче сыпучих материалов (объём);  
bulk_time_new = import_data(pth1[2], pth2[2]) #данные о подаче сыпучих материалов (время);  
gas_new = import_data(pth1[3], pth2[3]) #результаты измерения температуры;
temp_new = import_data(pth1[4], pth2[4]) #данные о проволочных материалах (объём);  
wire_time_new = import_data(pth1[4], pth2[4]) #данные о проволочных материалах (время)

In [7]:
arc_new.info()
arc_new.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14876 entries, 0 to 14875
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   key                   14876 non-null  int64  
 1   Начало нагрева дугой  14876 non-null  object 
 2   Конец нагрева дугой   14876 non-null  object 
 3   Активная мощность     14876 non-null  float64
 4   Реактивная мощность   14876 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 581.2+ KB


,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность
9773,2114,2019-07-28 01:13:27,2019-07-28 01:16:45,0.416684,0.347834
14866,3240,2019-09-06 15:25:31,2019-09-06 15:27:32,0.669469,0.497374
8336,1814,2019-07-09 20:16:56,2019-07-09 20:20:04,0.675231,0.501533
4883,1053,2019-06-12 23:29:10,2019-06-12 23:32:21,0.636881,0.501213
1923,416,2019-05-19 08:58:14,2019-05-19 09:08:36,0.901524,0.576528


In [8]:
bulk_new.info()
bulk_new.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   key      3129 non-null   int64  
 1   Bulk 1   252 non-null    float64
 2   Bulk 2   22 non-null     float64
 3   Bulk 3   1298 non-null   float64
 4   Bulk 4   1014 non-null   float64
 5   Bulk 5   77 non-null     float64
 6   Bulk 6   576 non-null    float64
 7   Bulk 7   25 non-null     float64
 8   Bulk 8   1 non-null      float64
 9   Bulk 9   19 non-null     float64
 10  Bulk 10  176 non-null    float64
 11  Bulk 11  177 non-null    float64
 12  Bulk 12  2450 non-null   float64
 13  Bulk 13  18 non-null     float64
 14  Bulk 14  2806 non-null   float64
 15  Bulk 15  2248 non-null   float64
dtypes: float64(15), int64(1)
memory usage: 391.3 KB


,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
1792,1854,NaN,NaN,231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,205.0,NaN,98.0,205.0
1075,1122,NaN,NaN,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,204.0,NaN,104.0,103.0
1127,1175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,NaN,202.0,104.0
541,570,NaN,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245.0,NaN
1665,1725,NaN,NaN,NaN,131.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310.0,NaN,191.0,209.0


In [9]:
bulk_time_new.info()
bulk_time_new.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   key      3129 non-null   int64 
 1   Bulk 1   252 non-null    object
 2   Bulk 2   22 non-null     object
 3   Bulk 3   1298 non-null   object
 4   Bulk 4   1014 non-null   object
 5   Bulk 5   77 non-null     object
 6   Bulk 6   576 non-null    object
 7   Bulk 7   25 non-null     object
 8   Bulk 8   1 non-null      object
 9   Bulk 9   19 non-null     object
 10  Bulk 10  176 non-null    object
 11  Bulk 11  177 non-null    object
 12  Bulk 12  2450 non-null   object
 13  Bulk 13  18 non-null     object
 14  Bulk 14  2806 non-null   object
 15  Bulk 15  2248 non-null   object
dtypes: int64(1), object(15)
memory usage: 391.3+ KB


,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
1174,1223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-06-18 16:33:17,NaN
1348,1400,NaN,NaN,2019-06-25 05:12:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-06-25 05:06:23,NaN,2019-06-25 05:01:01,2019-06-25 04:54:16
2512,2607,2019-08-14 16:26:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-14 16:20:30,NaN,2019-08-14 16:13:37,NaN
866,902,NaN,NaN,NaN,NaN,NaN,2019-06-07 08:47:00,NaN,NaN,NaN,NaN,2019-06-07 08:42:42,2019-06-07 08:30:28,NaN,2019-06-07 08:27:04,2019-06-07 08:24:00
1975,2045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 01:49:05,NaN,2019-07-25 01:49:05,2019-07-25 01:42:05


In [10]:
gas_new.info()
gas_new.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3239 entries, 0 to 3238
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   key     3239 non-null   int64  
 1   Газ 1   3239 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 50.7 KB


,key,Газ 1
483,486,19.213015
163,164,12.545387
600,603,12.335301
3150,3153,6.359977
1686,1689,33.597044


In [11]:
temp_new.info()
temp_new.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18092 entries, 0 to 18091
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   key           18092 non-null  int64  
 1   Время замера  18092 non-null  object 
 2   Температура   14665 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 424.2+ KB


,key,Время замера,Температура
15134,2707,2019-08-18 05:11:17,NaN
14878,2658,2019-08-16 15:38:21,NaN
16646,2971,2019-08-28 10:55:40,NaN
16,3,2019-05-03 12:35:57,1599.0
16019,2864,2019-08-24 05:18:15,NaN


In [12]:
wire_time_new.info()
wire_time_new.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18092 entries, 0 to 18091
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   key           18092 non-null  int64  
 1   Время замера  18092 non-null  object 
 2   Температура   14665 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 424.2+ KB


,key,Время замера,Температура
11596,2068,2019-07-26 00:01:58,1596.0
5564,990,2019-06-10 17:27:29,1581.0
13425,2402,2019-08-07 04:17:34,1587.0
5887,1047,2019-06-12 17:57:37,1612.0
14989,2679,2019-08-17 06:46:43,1564.0


# 3.Исследовательский анализ и предобработка данных

## 3.1 Предобработка данных

**arc_new**

Приведем наименования признаков в соответствие со snake_case

In [13]:
arc_new.columns = ['key', 'start_of_heating', 'end_of_heating', 'active_power', 'reactive_power']

In [14]:
arc_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14876 entries, 0 to 14875
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   key               14876 non-null  int64  
 1   start_of_heating  14876 non-null  object 
 2   end_of_heating    14876 non-null  object 
 3   active_power      14876 non-null  float64
 4   reactive_power    14876 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 581.2+ KB


Приведем время к date_time

In [15]:
arc_new.start_of_heating = pd.to_datetime(arc_new.start_of_heating, infer_datetime_format = True) 
arc_new.end_of_heating = pd.to_datetime(arc_new.end_of_heating, infer_datetime_format = True) 

/var/folders/5m/tbwsb3vs6734vbbw653my9v00000gp/T/ipykernel_14090/1436999925.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  arc_new.start_of_heating = pd.to_datetime(arc_new.start_of_heating, infer_datetime_format = True)
/var/folders/5m/tbwsb3vs6734vbbw653my9v00000gp/T/ipykernel_14090/1436999925.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  arc_new.end_of_heating = pd.to_datetime(arc_new.end_of_heating, infer_datetime_format = True)


In [16]:
arc_new.info()
arc_new.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14876 entries, 0 to 14875
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   key               14876 non-null  int64         
 1   start_of_heating  14876 non-null  datetime64[ns]
 2   end_of_heating    14876 non-null  datetime64[ns]
 3   active_power      14876 non-null  float64       
 4   reactive_power    14876 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(1)
memory usage: 581.2 KB


,key,start_of_heating,end_of_heating,active_power,reactive_power
7498,1631,2019-07-03 10:42:30,2019-07-03 10:45:35,0.766332,0.572820
11445,2499,2019-08-10 13:33:31,2019-08-10 13:38:43,0.700677,0.534767
8149,1769,2019-07-08 06:25:32,2019-07-08 06:27:48,0.603825,0.418942
12261,2662,2019-08-16 19:41:15,2019-08-16 19:43:15,0.252051,0.192502
457,98,2019-05-07 01:50:37,2019-05-07 01:51:52,0.410599,0.277339


Проверим пропуски

In [17]:
arc_new.isna().sum()

key                 0
start_of_heating    0
end_of_heating      0
active_power        0
reactive_power      0
dtype: int64

Пропусков нет

Проверим дубликаты

In [18]:
arc_new.duplicated().sum()

0

Явных дубликатов нет

In [19]:
arc_new.describe()

,key,start_of_heating,end_of_heating,active_power,reactive_power
count,14876.000000,14876,14876,14876.000000,14876.000000
mean,1615.220422,2019-07-05 12:25:51.921081088,2019-07-05 12:28:43.592027392,0.662752,0.438986
min,1.000000,2019-05-03 11:02:14,2019-05-03 11:06:02,0.223120,-715.479924
25%,806.000000,2019-06-03 23:18:23.249999872,2019-06-03 23:21:35,0.467115,0.337175
50%,1617.000000,2019-07-03 01:31:26.500000,2019-07-03 01:35:13,0.599587,0.441639
75%,2429.000000,2019-08-07 22:52:20.750000128,2019-08-07 22:56:47,0.830070,0.608201
max,3241.000000,2019-09-06 17:24:54,2019-09-06 17:26:15,1.463773,1.270284
std,934.571502,NaN,NaN,0.258885,5.873485


значения меньше 0 для реактивной мощности явно ошибочны

In [20]:
arc_new[arc_new.reactive_power > 0].count()

key                 14875
start_of_heating    14875
end_of_heating      14875
active_power        14875
reactive_power      14875
dtype: int64

In [21]:
arc_new = arc_new[arc_new.reactive_power > 0]

In [22]:
arc_new.describe()

,key,start_of_heating,end_of_heating,active_power,reactive_power
count,14875.000000,14875,14875,14875.000000,14875.000000
mean,1615.186756,2019-07-05 12:23:40.763025152,2019-07-05 12:26:32.438184960,0.662749,0.487115
min,1.000000,2019-05-03 11:02:14,2019-05-03 11:06:02,0.223120,0.153777
25%,806.000000,2019-06-03 23:13:02.500000,2019-06-03 23:16:13,0.467115,0.337190
50%,1617.000000,2019-07-03 01:28:12,2019-07-03 01:31:27,0.599578,0.441665
75%,2429.000000,2019-08-07 22:54:16.500000,2019-08-07 22:57:54,0.830135,0.608223
max,3241.000000,2019-09-06 17:24:54,2019-09-06 17:26:15,1.463773,1.270284
std,934.593897,NaN,NaN,0.258894,0.197612


**bulk_new**

Проверим пропуски

In [24]:
bulk_new.isna().sum()

key           0
Bulk 1     2877
Bulk 2     3107
Bulk 3     1831
Bulk 4     2115
Bulk 5     3052
Bulk 6     2553
Bulk 7     3104
Bulk 8     3128
Bulk 9     3110
Bulk 10    2953
Bulk 11    2952
Bulk 12     679
Bulk 13    3111
Bulk 14     323
Bulk 15     881
dtype: int64

Скорее всего пропуски связаны с тем, что в указанные плавки материалы не подавались (Объем 0). Логично заполнить данные пропуски 0

In [26]:
bulk_new = bulk_new.fillna(0)

In [27]:
bulk_new.isna().sum()

key        0
Bulk 1     0
Bulk 2     0
Bulk 3     0
Bulk 4     0
Bulk 5     0
Bulk 6     0
Bulk 7     0
Bulk 8     0
Bulk 9     0
Bulk 10    0
Bulk 11    0
Bulk 12    0
Bulk 13    0
Bulk 14    0
Bulk 15    0
dtype: int64

Приведем признаки к snake_case

In [28]:
bulk_new.columns = ['key', 'bulk_1', 'bulk_2', 'bulk_3', 'bulk_4', 
                    'bulk_5', 'bulk_6', 'bulk_7', 'bulk_8', 'bulk_9', 
                    'bulk_10', 'bulk_11', 'bulk_12', 'bulk_13', 'bulk_14', 'bulk_15']

In [30]:
bulk_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   key      3129 non-null   int64  
 1   bulk_1   3129 non-null   float64
 2   bulk_2   3129 non-null   float64
 3   bulk_3   3129 non-null   float64
 4   bulk_4   3129 non-null   float64
 5   bulk_5   3129 non-null   float64
 6   bulk_6   3129 non-null   float64
 7   bulk_7   3129 non-null   float64
 8   bulk_8   3129 non-null   float64
 9   bulk_9   3129 non-null   float64
 10  bulk_10  3129 non-null   float64
 11  bulk_11  3129 non-null   float64
 12  bulk_12  3129 non-null   float64
 13  bulk_13  3129 non-null   float64
 14  bulk_14  3129 non-null   float64
 15  bulk_15  3129 non-null   float64
dtypes: float64(15), int64(1)
memory usage: 391.3 KB


In [31]:
bulk_new.describe()

,key,bulk_1,bulk_2,bulk_3,bulk_4,bulk_5,bulk_6,bulk_7,bulk_8,bulk_9,bulk_10,bulk_11,bulk_12,bulk_13,bulk_14,bulk_15
count,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000,3129.000000
mean,1624.383509,3.160435,1.779163,47.240332,33.830617,2.633749,21.892298,2.441675,0.015660,0.463407,4.684564,4.345478,203.948226,1.041866,152.706616,115.319271
std,933.337642,11.869164,21.217878,74.239954,56.034505,20.918138,55.492582,31.944063,0.875978,6.154398,20.157323,22.699580,151.419639,14.114252,81.091421,84.488698
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,816.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,105.000000,0.000000,104.000000,0.000000
50%,1622.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,206.000000,0.000000,148.000000,106.000000
75%,2431.000000,0.000000,0.000000,80.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,263.000000,0.000000,202.000000,204.000000
max,3241.000000,185.000000,325.000000,454.000000,281.000000,603.000000,503.000000,772.000000,49.000000,147.000000,159.000000,313.000000,1849.000000,305.000000,636.000000,405.000000


Отрицательных значений нет

**bulk_time_new**

Приведем к snake_case

In [37]:
bulk_time_new.columns = ['key', 'bulk_1', 'bulk_2', 'bulk_3', 'bulk_4', 
                    'bulk_5', 'bulk_6', 'bulk_7', 'bulk_8', 'bulk_9', 
                    'bulk_10', 'bulk_11', 'bulk_12', 'bulk_13', 'bulk_14', 'bulk_15']

In [40]:
columns = ['bulk_1', 'bulk_2', 'bulk_3', 'bulk_4', 
                    'bulk_5', 'bulk_6', 'bulk_7', 'bulk_8', 'bulk_9', 
                    'bulk_10', 'bulk_11', 'bulk_12', 'bulk_13', 'bulk_14', 'bulk_15']

Приведем к date_time

In [41]:
def to_datetime(columns:str, df:pd.DataFrame) -> pd.DataFrame:
    for column in columns:
        df[column] = pd.to_datetime(df[column], infer_datetime_format = True) 
    return df

In [42]:
bulk_time_new = to_datetime(columns, bulk_time_new)

/var/folders/5m/tbwsb3vs6734vbbw653my9v00000gp/T/ipykernel_14090/3886688226.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[column] = pd.to_datetime(df[column], infer_datetime_format = True)
/var/folders/5m/tbwsb3vs6734vbbw653my9v00000gp/T/ipykernel_14090/3886688226.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[column] = pd.to_datetime(df[column], infer_datetime_format = True)
/var/folders/5m/tbwsb3vs6734vbbw653my9v00000gp/T/ipykernel_14090/3886688226.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed 

ValueError: time data "0" doesn't match format "%Y-%m-%d %H:%M:%S", at position 9. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [34]:
bulk_time_new.isna().sum()

key           0
Bulk 1     2877
Bulk 2     3107
Bulk 3     1831
Bulk 4     2115
Bulk 5     3052
Bulk 6     2553
Bulk 7     3104
Bulk 8     3128
Bulk 9     3110
Bulk 10    2953
Bulk 11    2952
Bulk 12     679
Bulk 13    3111
Bulk 14     323
Bulk 15     881
dtype: int64

Здесь аналогичная ситуация

In [35]:
bulk_time_new = bulk_time_new.fillna(0)